# Text Summarization Course from LinkedIn Learning
2024-04-11
https://www.linkedin.com/learning/ai-text-summarization-with-hugging-face/extractive-text-summarization

## Intent
Task: Adapt this during...
Hope: Use Stacks!

## References

- The Tasks site shows models, datasets and how to for each problem type: 
  https://huggingface.co/tasks/summarization
  

## Examples

1. Extractive Text Summarization:
   App: https://huggingface.co/spaces/issam9/sumy_space
   GitHub: https://github.com/miso-belica/sumy

2. Abstractive Text Summarization (NLP):
   https://huggingface.co/facebook/bart-large-cnn
   https://huggingface.co/google/pegasus-cnn_dailymail

3. Sequence-to-sequence models
   https://huggingface.co/google-t5/t5-small

## Process for this Notebook

1. Tutorial uses: https://colab.research.google.com but this now has a $/mo subscription and is no longer free..
2. 
   




## TODO Recording:

- First go to HuggingFace
- Under "Models" search for "T5 small" and show the model card page
- Go to Colab and log in (cloud.user@)
- Import this notebook from local machine
- Go to runtime and show that we are using the TPU
- After running pip install of libraries, please restart the kernel

Installing required huggingface libraries/modules

In [35]:
pip install transformers datasets evaluate rouge_score accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


> After the above, you will need to Kernal -> Restart Kernal to use updated packages.

Checking transformers version

> Note: if you see an error here "IProgress not found. Please update jupyter and ipywidgets", you might need to:
> `conda install -c conda-forge ipywidgets`

In [36]:
import transformers

transformers.__version__

'4.39.3'

## TODO Recording

- Go to the HuggingFace site
- Click on the Account on the top right -> Profile -> Settings -> Access Tokens
- Generate a new token and copy it
- Then run the cell below

> Note: this code will associate this Notebook with your HuggingFace account (Open Source Contrib)

In [37]:
# from huggingface_hub import notebook_login

# notebook_login()

https://huggingface.co/docs/datasets/v1.2.0/loading_datasets.html
Checking all datasets available in huggingface hub

In [38]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

DatasetInfo(id='acronym_identification', author=None, sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=765, likes=18, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'croissant', 'arxiv:2010.14678', 'region:us'], card_data=None, siblings=None)
DatasetInfo(id='ade_corpus_v2', author=None, sha='4ba01c71687dd7c996597042449448ea312126cf', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 42, 58, tzinfo=datetime.timezone.utc), private=False

Loading cnn daily mail dataset

In [43]:
from datasets import load_dataset

cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0")

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Dataset dictionary can be seen. As the dataset is quite large, we would be taking a subset of training data only.

In [44]:
cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0", split = "train[:3%]")

cnn_news_summary_ds

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 8613
})

Subset data's dimensions can be checked

In [45]:
cnn_news_summary_ds.shape

(8613, 3)

dataset description, features, homepage can be obtained
https://huggingface.co/docs/datasets/v1.2.0/exploring.html

In [46]:
print(cnn_news_summary_ds.description)

In [47]:
cnn_news_summary_ds.features

{'article': Value(dtype='string', id=None),
 'highlights': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [48]:
cnn_news_summary_ds.homepage

''

One row of example can be obtained

In [49]:
cnn_news_summary_ds[0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

Dataset is split into train and test sets

In [50]:
cnn_news_summary_ds = cnn_news_summary_ds.train_test_split(test_size = 0.2, seed = 42)

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

Text cleaning function is defined

In [51]:
def clean_txt(example):
    for txt in ["article", "highlights"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("\\", "")
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example


Both train and test sets are cleaned using 'map' method

In [52]:
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_txt)

cleaned_cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

Taking a look at a raw example

In [53]:
cnn_news_summary_ds["train"]["article"][7]

'(CNN)  -- Four-time defending champion Rafael Nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to Sweden\'s Robin Soderling at the French Open on Sunday. Soderling savors his unlikely triumph against Nadal in the French Open fourth round. Soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at Roland Garros and was a short-priced favorite to win a record fifth straight title. But Soderling never looked back after breezing through the first set of their fourth round match and closed out victory as Nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. The Spaniard tried to look on the positives after his shock defeat. "It\'s not a tragedy, losing here in Paris," he told the official tournament Web site www.rolandgarros.com . "It had to happen one day, and this is an excellent season for me. "Of course it\'s a bit sad, but I have to overcome this as quickly as possible. No one remembers defea

Now observing cleaned up version of the same raw data

In [54]:
cleaned_cnn_news_summary_ds["train"]["article"][7]

"(cnn)   four-time defending champion rafael nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to sweden's robin soderling at the french open on sunday. soderling savors his unlikely triumph against nadal in the french open fourth round. soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at roland garros and was a short-priced favorite to win a record fifth straight title. but soderling never looked back after breezing through the first set of their fourth round match and closed out victory as nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. the spaniard tried to look on the positives after his shock defeat. it's not a tragedy, losing here in paris, he told the official tournament web site www.rolandgarros.com . it had to happen one day, and this is an excellent season for me. of course it's a bit sad, but i have to overcome this as quickly as possible. no one remembers defeats on the

Similarly,both raw and cleaned up version of highlights(summary) of same example can be seen

In [55]:
cnn_news_summary_ds["train"]["highlights"][0]

'Maradona staying a clinic in Italy when tax authorities swooped .\nReports: Argentina coach owes millions in taxes from his time playing in Italy .\nMaradona says Serie A club should have paid the taxes .'

In [56]:
cleaned_cnn_news_summary_ds["train"]["highlights"][0]

'maradona staying a clinic in italy when tax authorities swooped .reports: argentina coach owes millions in taxes from his time playing in italy .maradona says serie a club should have paid the taxes .'

# Zero-Shot Learning
taking a random article for summarising

In [57]:
EXAMPLE_TEXT_INDEX = 33

example_text = cleaned_cnn_news_summary_ds["test"]["article"][EXAMPLE_TEXT_INDEX]

example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

In [58]:
MODEL_NAME = "t5-small"

In [59]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

# Results in Error: "Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors"
# This is because the T5-small model has a max sequence length of 512


Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever . lancey, a 10-week-old puppy, was hand-delivered to them earlier this week . he's the most human of any dog we've ever had, the company says ."}]

In [60]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)

summary_txt = summarizer(prefix + example_text)

summary_txt

[{'summary_text': 'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'}]

Gold/Reference summary is obtained

In [61]:
ref_txt = cleaned_cnn_news_summary_ds["test"]["highlights"][EXAMPLE_TEXT_INDEX]

ref_txt

"couple won auction to clone family dog, biotech company says .lancey is world's first commercially cloned dog, company says .dna of deceased dog sent to s. korea, and cloned puppy born november 18 .humane society says it's against commercial cloning of animals ."

Rouge metric is instantiated

In [62]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [63]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"

Rouge score with extra words in summary in comparison to reference(gold summary)

In [64]:
results = rouge.compute(predictions = [ex_1], references = [reference])

results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

Fewer extra words and better the score

In [65]:
results = rouge.compute(predictions = [ex_2], references = [reference])

results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

Coming back to our predicted summary

In [66]:
summary_txt[0]["summary_text"]

'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'

Rouge score for that text summary is obtained

In [67]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.13157894736842105,
 'rouge2': 0.027027027027027025,
 'rougeL': 0.07894736842105263,
 'rougeLsum': 0.07894736842105263}

All article texts and summaries are defined in a list

In [68]:
article_texts = cleaned_cnn_news_summary_ds["test"]["article"]

article_summaries = cleaned_cnn_news_summary_ds["test"]["highlights"]

Generating summaries (zero shot) for first 50 rows only as it will takes a very long time in summarizing whole set of examples

In [69]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [02:10<00:00,  2.62s/it]


Aggregated rouge scores are obtained

In [70]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.32629704621472744,
 'rouge2': 0.13921312842379102,
 'rougeL': 0.23932175088617413,
 'rougeLsum': 0.23967552995983654}

Here case by case rouge scores are obtained.

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

For the best rouge score, we can observe the predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(38, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(38, "Reference_summaries"))

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(12, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(12, "Reference_summaries"))

Loading  a T5 tokenizer to process text and summary

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Tokenizer is applied and token ids can be seen

In [ ]:
tokenizer(["four-time defending champion"])

In [ ]:
tokens = tokenizer.tokenize("four-time defending champion")

tokens

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

Loading T5 Model with AutoModelForSeq2SeqLM

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

The preprocessing function you want to create needs to:

Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
Use the keyword text_target argument when tokenizing labels.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["highlights"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

Applying the preprocessing function over the entire dataset by using 'map' method. We can speed up the map function by setting batched=True to process multiple elements of the dataset at once

In [ ]:
tokenized_cnn_news_summary_ds = cleaned_cnn_news_summary_ds.map(preprocess_function, batched = True)

tokenized_cnn_news_summary_ds

Now create a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}


Defining our training hyperparameters in Seq2SeqTrainingArguments. The only required parameter is output_dir which specifies where to save your model. We will push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the Trainer will evaluate the ROUGE metric and save the training checkpoint.
Passing the training arguments to Seq2SeqTrainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.
Calling train() to finetune your model.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "cnn_news_summary_model_trained_on_reduced_data",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_cnn_news_summary_ds["train"],
    eval_dataset = tokenized_cnn_news_summary_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

## TODO Recording:

- Immediately after training completes click on the link to the model on HuggingFace e.g. https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data
- Click on Model Card - should be nothing there
- Click on Files and Versions - should see the trained model files and some commits there
- Click on Training Metrics - that will take you to TensorBoard - show the graphs there (click on all tabs)

In [ ]:
trainer.push_to_hub()

## TODO Recording:

- Now go to the model on HuggingFace
- Click on "Model Card" show that it has been generated
- Click on "Files and Versions" and show

In [ ]:
example_text

The simplest way to try out your finetuned model for inference is to use it in a pipeline(). Instantiate a pipeline for summarization with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline(
    "summarization",
    model = "loony-user/cnn_news_summary_model_trained_on_reduced_data",
    truncation=True
)
summary_txt = summarizer(prefix + example_text)

summary_txt

Alternative way of summarising text(manual method)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("loony-user/cnn_news_summary_model_trained_on_reduced_data")

inputs = tokenizer(prefix + example_text, return_tensors = "pt", truncation=True).input_ids

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("loony-user/cnn_news_summary_model_trained_on_reduced_data")

outputs = model.generate(inputs, max_new_tokens = 100, do_sample = False)

tokenizer.decode(outputs[0], skip_special_tokens = True)

In [ ]:
ref_txt

In [ ]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = False)
result

In [ ]:
candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):

  candidate = summarizer(prefix + text)

  candidate_summaries.append(candidate[0]["summary_text"])

Looks like zero shot untuned model is having slighly better performance than our fine tuned model.

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50], use_stemmer = False)
result_agg

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

In [ ]:
result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(27, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(27, "Reference_summaries"))

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(23, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(23, "Reference_summaries"))